# Introducción

<font color=green>
    
La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes
utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el
comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline:

- Smart 
- Ultra

</font>

# Objetivo

<font color=green>
Desarrollar un modelo que escoja el plan correcto según el comportamiento de los clientes de Megaline.
    
Utilizando 3 tipos de modelos:

- Árbol de decisión
- Bosque aleatorio
- Regresión logística

Viendo y calibrando sus hiperparámetros para buscar la mayor exactitud posible. Siendo para este proyecto el umbral de exactitud es 0.75.
</font>


# Librerías y Datos 

## Librerías

In [1]:
import pandas as pd
from sklearn import set_config
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split 

## Datos

In [2]:
try:
    data = pd.read_csv('users_behavior.csv')
except:
    data = pd.read_csv('/datasets/users_behavior.csv')

### Exploración de Datos

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [5]:
data.tail()

,calls,minutes,messages,mb_used,is_ultra
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0
3213,80.0,566.09,6.0,29480.52,1


In [7]:
data['minutes'].max()

1632.0600000000004

### Descripción y observación de datos 

<font color=green>
Revisando la información de nuestro DataFrame, observamos lo siguiente:
    
- Tenemos un total de 5 columnas.
    - `сalls` — número de llamadas
    - `minutes` — duración total de la llamada en minutos
    - `messages` —número de mensajes de texto
    - `mb_used` — Tráfico de Internet utilizado en MB
    - `is_ultra` — plan para el mes actual (Ultra - 1, Smart - 0).
- No tiene datos faltantes.
- No tiene datos de formato equivocado.

No necesitamos corregir datos ni formatos.
</font>

## Preparación de los datos

<font color=green>
Como mencionado antes, en este caso no debemos llenar datos faltantes ni corregir el formato de estos. Pero de todas formas, debemos preparar los datos antes de poder trabajar con ellos. Para poder entrenar, validar y testear el modelo, debemos efectuar 2 pasos antes:
    
 1. Dividir los datos en distintos grupos. En este caso, dejaremos el 60% para entrenar, 20% para validar y 20% para testear el modelo.

 2. Crear los targets y features, considerando el entrenamiento, validación y testeo del modelo. Considerando la columna `is_ultra` como el objetivo(target).
</font>

In [6]:
#Separamos el grupo de testeo
data_rest, data_test = train_test_split(data, test_size=0.2, random_state=54321)
print(f'Tamaño data_rest:', data_rest.shape)
print('---------------------------')
print(f'Tamaño data_test:', data_test.shape)

Tamaño data_rest: (2571, 5)
---------------------------
Tamaño data_test: (643, 5)


In [7]:
#Separamos el grupo de entrenamiento y validación:
data_train, data_valid = train_test_split(data, test_size=0.25, random_state=54321)
print(f'Tamaño data_train:', data_train.shape)
print('----------------------------')
print(f'Tamaño data_valid:', data_valid.shape)

Tamaño data_train: (2410, 5)
----------------------------
Tamaño data_valid: (804, 5)


In [8]:
#Creamos los features y targets para nuestro modelo
features_train = data_train.drop(['is_ultra'], axis=1)
target_train = data_train['is_ultra']
features_valid = data_valid.drop(['is_ultra'], axis=1)
target_valid = data_valid['is_ultra']
features_test = data_test.drop(['is_ultra'], axis=1)
target_test = data_test['is_ultra']

In [9]:
#Imprimamos los tamaños finales de nuestros DataFrames
print(f'DataFrame de Entrenamiento:')
print(features_train.shape)
print(target_train.shape)
print('-------------------------')
print(f'DataFrame de Validación:')
print(features_valid.shape)
print(target_valid.shape)
print('---------------------')
print(f'DataFrame de Prueba:')
print(features_test.shape)
print(target_test.shape)

DataFrame de Entrenamiento:
(2410, 4)
(2410,)
-------------------------
DataFrame de Validación:
(804, 4)
(804,)
---------------------
DataFrame de Prueba:
(643, 4)
(643,)


# Modelos a ocupar

## Modelo: Árbol de decisión
<font color=green>
Para comenzar, utilizaremos el modelo de Árbol de decisión. Es el segundo modelo más rápido de procesamiento. Pero podemos jugar, cambiando su profundidad.    
</font>


In [10]:
best_tree=0
best_depth=0
score=0
for depth in range(1, 50):
    tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    tree.fit(features_train, target_train)
    score = tree.score(features_valid, target_valid)
    if score > best_tree:
        best_tree = score # guarda la mejor puntuación de accuracy en el conjunto de validación
        best_depth = depth # guarda el número de estimadores que corresponden a la mejor puntuación de accuracy

print("Accuracy del mejor modelo en el conjunto de validación (depth = {}): {}".format(best_depth, best_tree))


Accuracy del mejor modelo en el conjunto de validación (depth = 8): 0.7935323383084577


<font color=green>
    En el Árol de decisión, el resultado es que el mejor árbol es de una profundidad de 8, con una exactitud de 79.3%
</font>

In [11]:
#Árbol de decisión de regresión
best_model = None
best_result = 10000
best_depth = 0
for depth in range(1, 6): # selecciona el rango del hiperparámetro
    model = DecisionTreeRegressor(random_state=12345, max_depth=depth) # entrena el modelo en el conjunto de entrenamiento
    model.fit(features_train, target_train) # entrena el modelo en el conjunto de entrenamiento
    predictions_valid = model.predict(features_valid) # obtén las predicciones del modelo en el conjunto de validación
    result = mean_squared_error(target_valid, predictions_valid)**0.5 # calcula la RECM en el conjunto de validación
    if result < best_result:
        best_model = model
        best_result = result
        best_depth = depth

print(f"RECM del mejor modelo en el conjunto de validación (max_depth = {best_depth}): {best_result}")

RECM del mejor modelo en el conjunto de validación (max_depth = 5): 0.4010290103757847


<font color=green>
    En el Árol de decisión de regresión, el mejor resultado es el árbol de una profundidad de 5, con una exactitud de 40.1%
</font>

## Modelo: Bosque Aleatorio
<font color=green>
Seguimos con el Bosque Aleatorio. Que tiene la exactitud más alta, por su uso de un conjunto de árboles, pero a su vez, es el más lento, debendiendo de cuántos árboles tenga.    
</font>


In [12]:
#Bosque Aleatorio
best_score = 0
best_est = 0
best_depth = 0
for est in range(1, 20): # selecciona el rango del hiperparámetro
    for depth in range(1,20):
        model = RandomForestClassifier(random_state=54321, n_estimators=est) # configura el número de árboles
        model.fit(features_train, target_train) # entrena el modelo en el conjunto de entrenamiento
        score = model.score(features_valid, target_valid) # calcula la puntuación de accuracy en el conjunto de validación
        if score > best_score:
            best_score = score # guarda la mejor puntuación de accuracy en el conjunto de validación
            best_est = est # guarda el número de estimadores que corresponden a la mejor puntuación de accuracy
            best_depth = depth

print("Accuracy del mejor modelo en el conjunto de validación (n_estimators = {}): {}".format(best_est, best_score))

final_forest = RandomForestClassifier(random_state=54321, n_estimators=best_est, max_depth=best_depth) # cambia n_estimators para obtener el mejor modelo
final_forest.fit(features_train, target_train)

Accuracy del mejor modelo en el conjunto de validación (n_estimators = 16): 0.8022388059701493


RandomForestClassifier(max_depth=1, n_estimators=16, random_state=54321)

<font color=green>
En el Bosque aleatorio el mejor resultado con 16 árboles con una exactitud de 80.2%    
</font>

In [13]:
#Bosque aleatorio de regresión
best_model = None
best_result = 10000
best_est = 0
best_depth = 0
for est in range(1, 20):
    for depth in range (1, 20):
        model = RandomForestRegressor(random_state=12345, n_estimators=est, max_depth=depth) # inicializa el constructor de modelos con los parámetros random_state=12345 y n_estimators=est
        model.fit(features_train, target_train) # entrena el modelo en el conjunto de entrenamiento
        predictions_valid = model.predict(features_valid) # obtén las predicciones del modelo en el conjunto de validación
        result = mean_squared_error(target_valid, predictions_valid)**0.5 # calcula la RECM en el conjunto de validación
        if result < best_result:
            best_model = model
            best_result = result
            best_est = est
            best_depth = depth

print("RECM del mejor modelo en el conjunto de validación", best_result, "n_estimators:", best_est, "best_depth:", depth)

RECM del mejor modelo en el conjunto de validación 0.38540378423181687 n_estimators: 16 best_depth: 19


<font color=green>
En el Bosque aleatorio de regresión, el mejor resultado con 16 árboles, nos da una exactitud de 38.5%.   
</font>

## Modelo: Regresión logística
<font color=green>
Terminamos con la Regresión logística. Es el modelo más rápido, porque tiene el menor número de parámetros, y en exactitud es la intermedia de los tres modelos.    
</font>

In [14]:
model = LogisticRegression(random_state=54321, solver='liblinear') # inicializa el constructor de regresión logística con los parámetros random_state=54321 y solver='liblinear'
model.fit(features_train, target_train) # entrena el modelo en el conjunto de entrenamiento
score_train = model.score(features_train, target_train) # calcula la puntuación de accuracy en el conjunto de entrenamiento
score_valid = model.score(features_valid, target_valid) # calcula la puntuación de accuracy en el conjunto de validación

print("Accuracy del modelo de regresión logística en el conjunto de entrenamiento:", score_train)
print("Accuracy del modelo de regresión logística en el conjunto de validación:", score_valid)

Accuracy del modelo de regresión logística en el conjunto de entrenamiento: 0.7178423236514523
Accuracy del modelo de regresión logística en el conjunto de validación: 0.6940298507462687


<font color=green>
En la regresión logística, observamos que en el conjunto de entrenamiento, alcanza una exactitud de 71.7%, y en el conjunto de validación 69.4%   
</font>

In [15]:
#Regresión linear
model = LinearRegression() # inicializa el constructor de modelos
model.fit(features_train, target_train) # entrena el modelo en el conjunto de entrenamiento
predictions_valid = model.predict(features_valid) # obtén las predicciones del modelo en el conjunto de validación

result = mean_squared_error(target_valid, predictions_valid)**0.5# calcula la RECM en el conjunto de validación
print( "RECM del modelo de regresión lineal en el conjunto de validación:", result)

RECM del modelo de regresión lineal en el conjunto de validación: 0.4550494744660431


<font color=green>
En la regresión linear, observamos que en el conjunto de validación alcanza solamente el 45.5%   
</font>

# Testeo

<font color=green>
    
Según lo observado en los modelos, tenemos 2 que son bastantes cercanas en su exactitud, por lo tanto haremos el testo con cada uno de ellos. 
Primero entrenaremos los modelos con el `data_rest`. Una vez entrenado el modelo, lo compararemos con los datos de `data_test`.

</font>

In [16]:
features_rest = data_rest.drop(['is_ultra'], axis=1)
target_rest = data_rest['is_ultra']

In [17]:
#Árbol de decisión:

final_tree = DecisionTreeClassifier(max_depth=8, random_state=12345)
final_tree.fit(features_rest, target_rest)

score = final_tree.score(features_test, target_test)
print(score)



0.7776049766718507


<font color=green>
Ha bajado en 1.6% la exactitud del modelo Árboles de decisión (77.7%), en comparación con el de validación (79.3%).     
</font>

In [18]:
#Bosque aleatorio:

final_forest= RandomForestClassifier(max_depth=1, n_estimators=16, random_state=54321)
final_forest.fit(features_rest,target_rest)

score = final_forest.score(features_test, target_test)
print(score)


0.7262830482115086


<font color=green>
En el modelo de Bosque aleatorio (72.6%), ha bajado 7,6%, en comparación con el de validación (80.2%).
</font>

<font color=green>
Siendo el árbol de decisión, quién lleva una ventaja sobre el bosque aleatorio (5%). Pudiese haber otro método para poder aumentar la exactitud. Se intentó alterando alguno hiperparámetros, pero solamente se ha logrado obtener un modelo con esta exactitud.
</font>

# Conclusión

<font color=green>
Luego, de entrenar y validar nuestros 3 modelos, nos quedamos con dos posibles opciones:
    
    
- Árbol de decisión: con una profundidad máxima de 8 árboles y una exactitud de 77.7%

    - `final_tree = DecisionTreeClassifier(max_depth=8, random_state=12345)`
    
- Bosque aleatorio: Con un bosque de 16 árboles, una profundidad máxima de 1 y una exactitud de 72.6%

    - `final_forest= RandomForestClassifier(max_depth=1, n_estimators=16, random_state=54321`

</font>